In [1]:
import numpy as np
import os
import tensorflow as tf
import sys
import nest_asyncio

nest_asyncio.apply()

############ Tensforflow config ############
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

############## Import modules ##############
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

In [2]:
from modules import ucr_loader, utils
# get all datasets
root = '../../../data/'
dataset_dict = ucr_loader.get_datasets(root, prefix='**/')
# retrieve data
dataset_name = 'FordA'
trainX, trainY, testX, testY = ucr_loader.load_data(dataset_dict[dataset_name])
# preprocess data
trainX, trainY, testX, testY = ucr_loader.preprocess_data(trainX, trainY, testX, testY, normalize=False, standardize=True)
# additional preprocessing
trainX, trainY, valX, valY = utils.perform_datasplit(trainX, trainY, test_split=0.3)
n_classes = len(np.unique(trainY))
# Shapes
print('TrainX:', trainX.shape)
print('ValX:', valX.shape)
print('TestX:', testX.shape)
print('Classes:', n_classes)

# uncomment to use the save folders
#model_dir, result_dir = utils.maybe_create_dirs(dataset_name, root='../../../', dirs=['models', 'results'], return_paths=True, verbose=1)

TrainX: (2520, 500, 1)
ValX: (1081, 500, 1)
TestX: (3601, 500, 1)
Classes: 2


Using federated training to learn and achieve privacy

In [3]:
from modules.DM import trainer_federated

n_clients, use_stratified = 4, True

federated_train_pre, stratified_state = trainer_federated.make_federated_data(trainX, trainY, n_clients=n_clients, stratify=use_stratified, return_state=True)
federated_val_pre = trainer_federated.make_federated_data(valX, valY, n_clients=n_clients, stratify=use_stratified)

In [4]:
client_epochs, batch_size = 1, 32
federated_train = trainer_federated.preporcess_federated_data(federated_train_pre, num_epochs=client_epochs, batch_size=batch_size)
federated_val = trainer_federated.preporcess_federated_data(federated_val_pre, num_epochs=1, batch_size=batch_size)

In [5]:
adaptive, stepwise = True, False
server_learning_rate, client_learning_rate = 0.2, 0.2

federated_dict = {}
federated_dict['input_shape'] = trainX.shape[1:]
federated_dict['n_classes'] = n_classes
federated_dict['input_spec'] = federated_train[0].element_spec

federated_train_process, federated_eval_process = trainer_federated.create_federated_processes(federated_dict, server_learning_rate, client_learning_rate=client_learning_rate, adaptive=adaptive, stepwise=stepwise)
state = federated_train_process.initialize()

In [ ]:
parallel_clients = None

# uncomment to save the model
model_path = None#os.path.join(model_dir, 'AlexNet_1d_federated_batch-' + str(batch_size) + '_stratified-' + str(stratified_state)  + '_nclients-' + str(n_clients) + '_nparallel-' + str(parallel_clients) + '_adaptive-' + str(adaptive) + '.h5'
model_extern = trainer_federated.traintf(state, federated_dict, federated_train_process, federated_train_pre, 
                                          federated_eval_process=federated_eval_process, federated_val=federated_val, 
                                          n_clients=parallel_clients, epochs=100, batch_size=batch_size, lr_decay=adaptive, model_path=model_path, verbose=1)

In [ ]:
# uncomment to save the model
report_path = None#os.path.join(result_dir, 'AlexNet_federated_batch-' + str(batch_size) + '_stratified-' + str(stratified_state) + '_nclients-' + str(n_clients) + '_nparallel-' + str(parallel_clients) + '_adaptive-' + str(adaptive) + '.txt')
preds = np.argmax(model_extern.predict(testX, batch_size=batch_size, verbose=1), axis=1)
utils.compute_classification_report(testY, preds, save=report_path, verbose=1)